<a href="https://colab.research.google.com/github/sureshgv/simple_chatbot_for_website/blob/main/chat_bot_for_websites.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
!pip install langchain
!pip install faiss-cpu
!pip install openai
!pip install unstructured
!pip install langchain-community
!pip install html2text

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.3/400.3 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for python-libmagic: filename=python_libmagic-0.4.0-py3-none-any.whl size=4113 sha256=8249dd732d854e83f083b944a9d4e15c7aa73a925a34b6484a3592b76e42b78e
  Stored in directory: /root/.cache/pip/wheels/9b/e1/b4/e6bd850c514b4d151b06892c3d75d06be12c82b9f4036ec450
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for cffi
  Running setup.py clean for cffi
Successfully built python-libmagic
Failed to build cffi
ERROR: Could not build wheels for cffi, which is required to install pyproject.toml-based projects


In [20]:
urls = ["https://lmsys.org/blog/2023-03-30-vicuna/", "https://www.meetup.com/ai-user-group/events/299617496/"]

In [21]:
from langchain.document_loaders import UnstructuredURLLoader
loaders = UnstructuredURLLoader(urls=urls)
data = loaders.load()

In [22]:
data

[Document(page_content='LMSYS ORG\n\nProjects\n\nBlog\n\nAbout\n\nDonations\n\nChatbot Arena\n\nProjects\n\nBlog\n\nAbout\n\nDonations\n\nChatbot Arena\n\nVicuna: An Open-Source Chatbot Impressing GPT-4 with 90%* ChatGPT Quality\n\nby: The Vicuna Team, Mar 30, 2023\n\nWe introduce Vicuna-13B, an open-source chatbot trained by fine-tuning LLaMA on user-shared conversations collected from ShareGPT. Preliminary evaluation using GPT-4 as a judge shows Vicuna-13B achieves more than 90%* quality of OpenAI ChatGPT and Google Bard while outperforming other models like LLaMA and Stanford Alpaca in more than 90%* of cases. The cost of training Vicuna-13B is around $300. The code and weights, along with an online demo, are publicly available for non-commercial use.\n\nVicuna (generated by stable diffusion 2.1)\n\nAccording to a fun and non-scientific evaluation with GPT-4. Further rigorous evaluation is needed.\n\nHow Good is Vicuna?\n\nAfter fine-tuning Vicuna with 70K user-shared ChatGPT conver

In [29]:
from langchain_community.document_loaders import AsyncHtmlLoader

urls = ["https://www.meetup.com/ai-user-group/events/299617496/"]
loader = AsyncHtmlLoader(urls)
docs = loader.load()

Fetching pages: 100%|##########| 1/1 [00:00<00:00, 13.09it/s]


In [30]:
!pip install html2text

In [31]:
from langchain_community.document_transformers import Html2TextTransformer

html2text = Html2TextTransformer()
docs_transformed = html2text.transform_documents(docs)
docs_transformed

[Document(page_content="Skip to content\n\n# AI For Developers #14\n\nHosted By\n\nCyril A. and Jackson B.\n\n## Details\n\nDevelopers, mark your calendars for May 29! Join us at our monthly AI for\nDevelopers event to dive deep into AI development, machine learning, deep\nlearning, natural language processing, computer vision, AI programming, and AI\nalgorithms.\n\nDiscover the power of AI technologies like ChatGPT and Codex to streamline\nyour development process and create innovative solutions.\n\nDon't miss this chance to network with fellow developers, learn about the\nlatest AI trends, and boost your skills in Generative AI development!\n\n**Speakers**\n\n  * Adam Silverman | Founder - AgentOps\n  * Justin Strong | Founder - DeepUnitAI\n  * Vinod Iyengar | VP of Product & Marketing - ThirdAI\n\n**Agenda**\n\n  * **6pm:** Doors Open\n  * **6:30pm:** Build Compliant AI Agents With Observability, Evals, and Replay Analytics _by Adam Silverman_\n  * **7pm:** Autonomously Generate Uni

In [37]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(separator='\n',
                                      chunk_size = 1000,
                                      chunk_overlap = 200)

docs = text_splitter.split_documents(docs_transformed)

docs

[Document(page_content="Skip to content\n# AI For Developers #14\nHosted By\nCyril A. and Jackson B.\n## Details\nDevelopers, mark your calendars for May 29! Join us at our monthly AI for\nDevelopers event to dive deep into AI development, machine learning, deep\nlearning, natural language processing, computer vision, AI programming, and AI\nalgorithms.\nDiscover the power of AI technologies like ChatGPT and Codex to streamline\nyour development process and create innovative solutions.\nDon't miss this chance to network with fellow developers, learn about the\nlatest AI trends, and boost your skills in Generative AI development!\n**Speakers**\n  * Adam Silverman | Founder - AgentOps\n  * Justin Strong | Founder - DeepUnitAI\n  * Vinod Iyengar | VP of Product & Marketing - ThirdAI\n**Agenda**\n  * **6pm:** Doors Open\n  * **6:30pm:** Build Compliant AI Agents With Observability, Evals, and Replay Analytics _by Adam Silverman_\n  * **7pm:** Autonomously Generate Unit Tests _by Justin Str

In [39]:
len(docs)

3

In [43]:
import pickle
import faiss
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
import os

embeddings = OpenAIEmbeddings()


In [44]:
embeddings

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x7bf9906d7d60>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x7bf990701ed0>, model='text-embedding-ada-002', deployment='text-embedding-ada-002', openai_api_version='', openai_api_base=None, openai_api_type='', openai_proxy='', embedding_ctx_length=8191, openai_api_key='sk-proj-Q73BBZfePPkGwAw7mI1eT3BlbkFJ2vYShjcCx0nUskIwH2DK', openai_organization=None, allowed_special=set(), disallowed_special='all', chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None)

In [46]:
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 11.9 MB/s eta 0:00:00


In [48]:
# Dump every thing locally
vectoreStore_openAPI = FAISS.from_documents(docs, embeddings)

with open("faiss_store_openai.pkl", "wb") as f:
  pickle.dump(vectoreStore_openAPI, f)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [49]:
# Read the vector store from local env
with open("faiss_store_openai.pkl", "rb") as f:
  VectoreStore = pickle.load(f)

FileNotFoundError: [Errno 2] No such file or directory: 'faiss_store_openai.pkl'

In [ ]:
VectoreStore

In [ ]:
llm = OpenAI(temparature=0)
llm

In [ ]:
chain = RetrievalQAWithSourcesChain.from_llm(llm=llm, retriever=VestoreStore.as_retriever())


In [ ]:
chain({"question": "When is the AI for Developers meet up?", return_only_outputs=True})